<a href="https://colab.research.google.com/github/ikhwanafif05/Automated-Valuation-Engine/blob/main/Valuation_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# --- AUTOMATED COMPS ENGINE ---
# Installs necessary libraries
!pip install yfinance pandas openpyxl xlsxwriter

import yfinance as yf
import pandas as pd
import numpy as np

In [4]:
def build_commercial_comps(tickers):
    """
    Fetches data, calculates multiples, and adds Peer Group Averages.
    """
    print(f"🚀 Launching Commercial Valuation Engine for: {tickers}")
    data = []

    for ticker in tickers:
        try:
            stock = yf.Ticker(ticker)
            info = stock.info

            # COMMERCIAL METRICS (The stuff that matters in a Pitch Book)
            metrics = {
                'Ticker': ticker,
                'Company': info.get('shortName', 'N/A'),
                'Price': info.get('currentPrice', np.nan),
                'Mkt Cap ($B)': info.get('marketCap', 0) / 1e9,
                'Ent Value ($B)': info.get('enterpriseValue', 0) / 1e9,
                'EV/Revenue': info.get('enterpriseToRevenue', np.nan),
                'EV/EBITDA': info.get('enterpriseToEbitda', np.nan),
                'P/E (LTM)': info.get('trailingPE', np.nan),
                'P/E (Fwd)': info.get('forwardPE', np.nan),
                'Net Debt/EBITDA': info.get('debtToEquity', 0) / 100 # Proxy using D/E (imperfect but automated)
            }
            data.append(metrics)
            print(f"   ✅ Analyzed: {ticker}")
        except:
            print(f"   ❌ Error: {ticker}")

    df = pd.DataFrame(data)

    # --- THE BANKER'S TOUCH: SUMMARY STATISTICS ---
    # We must calculate Mean and Median so the MD has a benchmark.
    numeric_cols = ['Price', 'Mkt Cap ($B)', 'Ent Value ($B)', 'EV/Revenue', 'EV/EBITDA', 'P/E (LTM)', 'P/E (Fwd)']

    # Calculate Mean & Median
    mean_row = df[numeric_cols].mean()
    median_row = df[numeric_cols].median()

    # Append them to the bottom (The "Football Field" benchmark)
    mean_row['Company'] = 'PEER MEAN'
    median_row['Company'] = 'PEER MEDIAN'
    mean_row['Ticker'] = '---'
    median_row['Ticker'] = '---'

    # Append to DataFrame using proper concatenation
    summary_df = pd.DataFrame([mean_row, median_row])
    df_final = pd.concat([df, summary_df], ignore_index=True)

    return df_final

def export_client_ready_excel(df, filename="BofA_Comps_Output.xlsx"):
    """
    Exports to Excel with CONDITIONAL FORMATTING.
    Green = Cheap (Low Multiple), Red = Expensive (High Multiple).
    """
    writer = pd.ExcelWriter(filename, engine='xlsxwriter')
    df.to_excel(writer, sheet_name='Comps Analysis', index=False)

    workbook = writer.book
    worksheet = writer.sheets['Comps Analysis']

    # 1. FORMATTING DEFINITIONS
    money_fmt = workbook.add_format({'num_format': '$#,##0.00'})
    multiple_fmt = workbook.add_format({'num_format': '0.0x'}) # e.g. 15.2x
    header_fmt = workbook.add_format({'bold': True, 'bottom': 2, 'bg_color': '#D9D9D9'})

    # 2. APPLY FORMATS TO COLUMNS
    worksheet.set_column('C:E', 15, money_fmt)    # Price, Mkt Cap, EV
    worksheet.set_column('F:J', 12, multiple_fmt) # Multiples

    # 3. CONDITIONAL FORMATTING (The "Heatmap")
    # Apply to EV/EBITDA and P/E columns (F to I)
    # Green = Low (Good/Cheap), Red = High (Expensive)
    # Note: Rows vary, assuming data ends before summary stats (dynamic range)
    data_rows = len(df) - 2 # Exclude Mean/Median

    # Green-Red Scale for Multiples
    worksheet.conditional_format(1, 5, data_rows, 8, { # Columns F to I (Indices 5-8)
        'type': '3_color_scale',
        'min_color': '#63BE7B', # Green (Low Multiple)
        'mid_color': '#FFFFFF', # White
        'max_color': '#F8696B'  # Red (High Multiple)
    })

    print(f"\n💾 EXPORT COMPLETE: {filename}")
    print("   -> Opened Excel writer")
    print("   -> Applied 'Banker' formatting (Currency, Multiples)")
    print("   -> Applied Heatmap (Green=Cheap, Red=Expensive)")

    writer.close()

# --- EXECUTION ---
# Let's run a Banking Sector Test for BofA
banking_peers = ['BAC', 'JPM', 'C', 'WFC', 'GS', 'MS']

# 1. Build Data
df_comps = build_commercial_comps(banking_peers)

# 2. Display Raw Table
print("\n📊 RAW DATA PREVIEW:")
display(df_comps)

# 3. Create the "Weaponized" Excel
export_client_ready_excel(df_comps)

🚀 Launching Commercial Valuation Engine for: ['BAC', 'JPM', 'C', 'WFC', 'GS', 'MS']
   ✅ Analyzed: BAC
   ✅ Analyzed: JPM
   ✅ Analyzed: C
   ✅ Analyzed: WFC
   ✅ Analyzed: GS
   ✅ Analyzed: MS

📊 RAW DATA PREVIEW:


,Ticker,Company,Price,Mkt Cap ($B),Ent Value ($B),EV/Revenue,EV/EBITDA,P/E (LTM),P/E (Fwd),Net Debt/EBITDA
0,BAC,Bank of America Corporation,55.950000,414.418698,423.431635,4.174000,NaN,15.286885,12.841228,0.00000
1,JPM,JP Morgan Chase & Co.,325.480000,894.989894,643.135963,3.846000,NaN,16.128841,15.366225,0.00000
2,C,"Citigroup, Inc.",118.700000,218.514571,48.492892,0.643000,NaN,16.671349,12.061783,0.00000
3,WFC,Wells Fargo & Company,95.200000,304.967582,323.684860,4.094000,NaN,15.683689,13.726618,0.00000
4,GS,"Goldman Sachs Group, Inc. (The)",914.340000,276.790018,23.209814,0.405000,NaN,18.587925,16.567950,5.86142
5,MS,Morgan Stanley,181.900000,290.373468,196.185358,2.863000,NaN,18.656410,17.219460,4.20553
6,---,PEER MEAN,281.928333,400.009039,276.356754,2.670833,NaN,16.835850,14.630544,NaN
7,---,PEER MEDIAN,150.300000,297.670525,259.935109,3.354500,NaN,16.400095,14.546422,NaN



💾 EXPORT COMPLETE: BofA_Comps_Output.xlsx
   -> Opened Excel writer
   -> Applied 'Banker' formatting (Currency, Multiples)
   -> Applied Heatmap (Green=Cheap, Red=Expensive)


In [7]:
def build_commercial_comps(tickers):
    print(f"🚀 Launching Commercial Valuation Engine for: {tickers}")
    data = []

    for ticker in tickers:
        try:
            stock = yf.Ticker(ticker)
            info = stock.info

            # SAFEGUARD: If 'debtToEquity' is missing, treat as NaN
            debt_equity = info.get('debtToEquity', np.nan)
            net_debt_ebitda = debt_equity / 100 if debt_equity is not None else np.nan

            metrics = {
                'Ticker': ticker,
                'Company': info.get('shortName', 'N/A'),
                'Price': info.get('currentPrice', np.nan),
                'Mkt Cap ($B)': info.get('marketCap', 0) / 1e9,
                'Ent Value ($B)': info.get('enterpriseValue', 0) / 1e9,
                'EV/Revenue': info.get('enterpriseToRevenue', np.nan),
                'EV/EBITDA': info.get('enterpriseToEbitda', np.nan),
                'P/E (LTM)': info.get('trailingPE', np.nan),
                'P/E (Fwd)': info.get('forwardPE', np.nan),
                'Net Debt/EBITDA': net_debt_ebitda
            }
            data.append(metrics)
            print(f"   ✅ Analyzed: {ticker}")
        except:
            print(f"   ❌ Error: {ticker}")

    df = pd.DataFrame(data)

    # --- FIX: Ensure we only average columns that actually exist ---
    # We include 'Net Debt/EBITDA' in the calculation list now.
    numeric_cols = ['Price', 'Mkt Cap ($B)', 'Ent Value ($B)', 'EV/Revenue', 'EV/EBITDA', 'P/E (LTM)', 'P/E (Fwd)', 'Net Debt/EBITDA']

    # Calculate Mean & Median (ignoring NaNs automatically)
    mean_row = df[numeric_cols].mean(skipna=True)
    median_row = df[numeric_cols].median(skipna=True)

    # Create Summary DataFrame
    mean_row['Company'] = 'PEER MEAN'
    median_row['Company'] = 'PEER MEDIAN'
    mean_row['Ticker'] = '---'
    median_row['Ticker'] = '---'

    # Reconstruct the Summary Rows to match the main DataFrame structure
    summary_df = pd.DataFrame([mean_row, median_row])

    # Combine
    df_final = pd.concat([df, summary_df], ignore_index=True)

    return df_final

def export_client_ready_excel(df, filename="BofA_Comps_Output.xlsx"):
    writer = pd.ExcelWriter(filename, engine='xlsxwriter')
    df.to_excel(writer, sheet_name='Comps Analysis', index=False)

    workbook = writer.book
    worksheet = writer.sheets['Comps Analysis']

    # FORMATTING
    money_fmt = workbook.add_format({'num_format': '$#,##0.00'})
    multiple_fmt = workbook.add_format({'num_format': '0.0x'})

    worksheet.set_column('C:E', 15, money_fmt)
    worksheet.set_column('F:J', 12, multiple_fmt)

    # CONDITIONAL FORMATTING (Heatmap)
    data_rows = len(df) - 2
    worksheet.conditional_format(1, 5, data_rows, 8, {
        'type': '3_color_scale',
        'min_color': '#63BE7B',
        'mid_color': '#FFFFFF',
        'max_color': '#F8696B'
    })

    print(f"\n💾 EXPORT COMPLETE: {filename}")
    writer.close()

# --- EXECUTION ---
# CHANGED SECTOR: We use Beverages to guarantee EV/EBITDA exists
# (Banks don't use EBITDA, so it returns NaN)
consumer_peers = ['KO', 'PEP', 'MNST', 'KDP'] # Coke, Pepsi, Monster, Keurig

# 1. Build Data
df_comps = build_commercial_comps(consumer_peers)

# 2. Display Raw Table
print("\n📊 RAW DATA PREVIEW (Beverages):")
display(df_comps)

# 3. Export
export_client_ready_excel(df_comps)

🚀 Launching Commercial Valuation Engine for: ['KO', 'PEP', 'MNST', 'KDP']
   ✅ Analyzed: KO
   ✅ Analyzed: PEP
   ✅ Analyzed: MNST
   ✅ Analyzed: KDP

📊 RAW DATA PREVIEW (Beverages):


,Ticker,Company,Price,Mkt Cap ($B),Ent Value ($B),EV/Revenue,EV/EBITDA,P/E (LTM),P/E (Fwd),Net Debt/EBITDA
0,KO,Coca-Cola Company (The),69.12,297.469477,331.727208,6.96000,20.343,22.887419,21.459574,1.447710
1,PEP,"Pepsico, Inc.",142.23,194.723856,236.819792,2.56400,13.917,27.039923,16.602543,2.601900
2,MNST,Monster Beverage Corporation,76.16,74.409935,71.890928,9.01400,28.899,43.272728,34.024303,0.007790
3,KDP,Keurig Dr Pepper Inc.,27.73,37.673509,55.787512,3.44900,12.977,23.905172,12.770386,0.735670
4,---,PEER MEAN,78.81,151.069194,174.056360,5.49675,19.034,29.276311,21.214202,1.198267
5,---,PEER MEDIAN,72.64,134.566896,154.355360,5.20450,17.130,25.472548,19.031059,1.091690



💾 EXPORT COMPLETE: BofA_Comps_Output.xlsx


In [6]:
# --- EXECUTION: BIG TECH (THE MONEY SHOT) ---
# We use the giants. This is what moves the S&P 500.
tech_peers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'NVDA', 'TSLA']

# 1. Build Data
df_tech = build_commercial_comps(tech_peers)

# 2. Display Raw Table (The "Scouting Report")
print("\n📊 RAW DATA PREVIEW (Big Tech):")
display(df_tech)

# 3. Export to Excel (The "Client Deliverable")
# naming it 'Big_Tech_Valuation.xlsx' so it looks specific
export_client_ready_excel(df_tech, filename="Big_Tech_Valuation.xlsx")

🚀 Launching Commercial Valuation Engine for: ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'NVDA', 'TSLA']
   ✅ Analyzed: AAPL
   ✅ Analyzed: MSFT
   ✅ Analyzed: GOOGL
   ✅ Analyzed: AMZN
   ✅ Analyzed: META
   ✅ Analyzed: NVDA
   ✅ Analyzed: TSLA

📊 RAW DATA PREVIEW (Big Tech):


,Ticker,Company,Price,Mkt Cap ($B),Ent Value ($B),EV/Revenue,EV/EBITDA,P/E (LTM),P/E (Fwd),Net Debt/EBITDA
0,AAPL,Apple Inc.,271.010000,4021.894250,4062.219338,9.761000,28.064000,36.377182,29.602146,1.524110
1,MSFT,Microsoft Corporation,472.940000,3515.441742,3533.431636,12.026000,21.230000,33.661210,25.233720,0.331540
2,GOOGL,Alphabet Inc.,315.150000,3817.135145,3748.614111,9.725000,25.822000,31.141304,28.149628,0.114240
3,AMZN,"Amazon.com, Inc.",226.500000,2421.333950,2487.574856,3.598000,17.807000,31.946404,28.871672,0.434050
4,META,"Meta Platforms, Inc.",650.410000,1639.376355,1645.988413,8.688000,16.728000,28.817455,21.382032,0.263110
5,NVDA,NVIDIA Corporation,188.850000,4597.931311,4539.269251,24.256000,40.279000,46.745052,24.959162,0.091020
6,TSLA,"Tesla, Inc.",438.070000,1456.941564,1429.828665,14.951000,132.785000,302.117250,195.353300,0.170820
7,---,PEER MEAN,366.132857,3067.150617,3063.846610,11.857857,40.387857,72.972265,50.507380,0.418413
8,---,PEER MEDIAN,315.150000,3515.441742,3533.431636,9.761000,25.822000,33.661210,28.149628,0.263110



💾 EXPORT COMPLETE: Big_Tech_Valuation.xlsx
